In [1]:
import pandas as pd

# Interacting with the API
import json
from ibm_watson import NaturalLanguageUnderstandingV1
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
from ibm_watson.natural_language_understanding_v1 import Features, EmotionOptions, SentimentOptions, CategoriesOptions

In [2]:
creds = {
  "apikey": "DxXZG9G24IFMBznqOQcI3SYa7MW6kU-hj3Hb3cWa-CoI",
  "iam_apikey_description": "Auto-generated for key 4a896ef7-db2c-4b9a-ba6e-87b526989ec2",
  "iam_apikey_name": "Auto-generated service credentials",
  "iam_role_crn": "crn:v1:bluemix:public:iam::::serviceRole:Manager",
  "iam_serviceid_crn": "crn:v1:bluemix:public:iam-identity::a/9cff25d774234ca6926e478038b1a5f0::serviceid:ServiceId-66f813a7-b446-48b3-838f-48fd95e3f20c",
  "url": "https://api.us-south.natural-language-understanding.watson.cloud.ibm.com/instances/60e2504b-0a72-434f-b0a6-606bca97a695"
}

In [14]:
# data
player_text = pd.read_csv('data/player_text_by_season.csv')
player_text.head()

,year,twitter_handle,cleaned_text
0,2014-2015,alexlen,great time visiting hq today and seeing new ga...
1,2012-2013,alexlen,great time last night meeting wonderful people...
2,2016-2017,alexlen,come see me tonight at pm on camelback &amp; l...
3,2019-2020,alexlen,": theres no doubt in my mind, though. im the s..."
4,2015-2016,alexlen,: [new video] : couple days from last week wit...


In [25]:
authenticator = IAMAuthenticator(creds['apikey'])
natural_language_understanding = NaturalLanguageUnderstandingV1(
    version='2020-08-01',
    authenticator=authenticator
)
natural_language_understanding.set_service_url(creds['url'])

In [5]:
# doc_labels
#   doc_labels
#    ' '.join([d['label'] for d in response['categories']])
#   doc_labels_weights
#    ' '.join([str(d['score']) for d in response['categories']])
# emotions
#   emot_anger
#    response['emotion']['document']['emotion']['anger']
#   emot_disgust
#    response['emotion']['document']['emotion']['disgust']
#   emot_fear
#    response['emotion']['document']['emotion']['fear']
#   emot_joy
#    response['emotion']['document']['emotion']['joy']
#   emot_sadness
#    response['emotion']['document']['emotion']['sadness']
# sentiment
#   sent_class
#    response['sentiment']['document']['label']
#   sent_score
#    response['sentiment']['document']['score']


In [6]:
def get_nlu_score(handle,text,season,nlu):
    nlu_scores = pd.DataFrame(columns=['twitter_handle','season','cleaned_text',
                                      'doc_labels','doc_labels_weights',
                                      'emot_anger','emot_disgust','emot_fear',
                                      'emot_joy','emot_sadness',
                                      'sent_class','sent_score'])
    
    response = nlu.analyze(text=text,
                           features=Features(emotion=EmotionOptions(document=True),
                           sentiment=SentimentOptions(document=True),
                           categories=CategoriesOptions(limit=20))).get_result()
    
    row = [handle,season,text,
          ' '.join([d['label'] for d in response['categories']]),
          ' '.join([str(d['score']) for d in response['categories']]),
          response['emotion']['document']['emotion']['anger'],
          response['emotion']['document']['emotion']['disgust'],
          response['emotion']['document']['emotion']['fear'],
          response['emotion']['document']['emotion']['joy'],
          response['emotion']['document']['emotion']['sadness'],
          response['sentiment']['document']['label'],
          response['sentiment']['document']['score']]
    
    nlu_scores.loc[0] = row
    return nlu_scores

In [30]:
def get_all_nlu_scores(texts_df,nlu):
    nlu_scores = pd.DataFrame(columns=['twitter_handle','season','cleaned_text',
                                      'doc_labels','doc_labels_weights',
                                      'emot_anger','emot_disgust','emot_fear',
                                      'emot_joy','emot_sadness',
                                      'sent_class','sent_score'])
    error_rows = pd.DataFrame(columns=['twitter_handle',
                                      'cleaned_text',
                                      'year'])
    error_num = 0
    for i in range(len(texts_df)):
        try:
            row = get_nlu_score(texts_df['twitter_handle'].iloc[i],
                               texts_df['cleaned_text'].iloc[i],
                               texts_df['year'].iloc[i],
                               nlu)
            nlu_scores = pd.concat([nlu_scores,row])
        except:
            print('Error occurred. Skipping user '+str(texts_df['twitter_handle'].iloc[i]))
            error_row = [texts_df['twitter_handle'].iloc[i],
                                    texts_df['cleaned_text'].iloc[i],
                                    texts_df['year'].iloc[i]]
            error_rows[error_num] = error_row
            error_num += 1
    return nlu_scores, error_rows, error_num

In [31]:
nlu_scores, error_rows, error_num = get_all_nlu_scores(player_text,natural_language_understanding)

Error occurred. Skipping user GorguiDieng
Error occurred. Skipping user juanchiviris41
Error occurred. Skipping user juanchiviris41
Error occurred. Skipping user juanchiviris41
Error occurred. Skipping user juanchiviris41
Error occurred. Skipping user juanchiviris41
Error occurred. Skipping user juanchiviris41
Error occurred. Skipping user juanchiviris41
Error occurred. Skipping user juanchiviris41


ERROR:root:unsupported text language: tr
Traceback (most recent call last):
  File "/home/mparkerseegmiller/.local/lib/python3.5/site-packages/ibm_cloud_sdk_core/base_service.py", line 246, in send
    response.status_code, http_response=response)
ibm_cloud_sdk_core.api_exception.ApiException: Error: unsupported text language: tr, Code: 400 , X-global-transaction-id: fd78f579-2dd8-4022-825b-89248abed341


Error occurred. Skipping user FurkanKorkmaz


ERROR:root:unsupported text language: tr
Traceback (most recent call last):
  File "/home/mparkerseegmiller/.local/lib/python3.5/site-packages/ibm_cloud_sdk_core/base_service.py", line 246, in send
    response.status_code, http_response=response)
ibm_cloud_sdk_core.api_exception.ApiException: Error: unsupported text language: tr, Code: 400 , X-global-transaction-id: f1650d2e-4316-4b98-9902-35320cebd7f1


Error occurred. Skipping user FurkanKorkmaz


...

Error occurred. Skipping user satoransky


ERROR:root:unsupported text language: cs
Traceback (most recent call last):
  File "/home/mparkerseegmiller/.local/lib/python3.5/site-packages/ibm_cloud_sdk_core/base_service.py", line 246, in send
    response.status_code, http_response=response)
ibm_cloud_sdk_core.api_exception.ApiException: Error: unsupported text language: cs, Code: 400 , X-global-transaction-id: 6bd308ec-9d08-4008-b750-7963a2a3f849


Error occurred. Skipping user satoransky
Error occurred. Skipping user satoransky
Error occurred. Skipping user satoransky
Error occurred. Skipping user KyleSingler
Error occurred. Skipping user KyleSingler
Error occurred. Skipping user luka7doncic
Error occurred. Skipping user luka7doncic
Error occurred. Skipping user luka7doncic
Error occurred. Skipping user luka7doncic
Error occurred. Skipping user luka7doncic
Error occurred. Skipping user luka7doncic
Error occurred. Skipping user DTP_Mario
Error occurred. Skipping user DTP_Mario
Error occurred. Skipping user DTP_Mario
Error occurred. Skipping user DTP_Mario
Error occurred. Skipping user DTP_Mario
Error occurred. Skipping user DTP_Mario
Error occurred. Skipping user DTP_Mario
Error occurred. Skipping user DTP_Mario


ERROR:root:not enough text for language id
Traceback (most recent call last):
  File "/home/mparkerseegmiller/.local/lib/python3.5/site-packages/ibm_cloud_sdk_core/base_service.py", line 246, in send
    response.status_code, http_response=response)
ibm_cloud_sdk_core.api_exception.ApiException: Error: not enough text for language id, Code: 422 , X-global-transaction-id: 5a6505be-a485-474a-bbfb-ca85c2f595bd


Error occurred. Skipping user Khyri_2


In [33]:
print(str(error_num)+' errors occured out of '+str(len(nlu_scores))+' total seasons of data.')

259 errors occured out of 2709 total seasons of data.


In [35]:
nlu_scores.to_csv('data/player_by_season_nlu_scores.csv',index=False)